In [8]:
import numpy as np
import pandas as pd

In [9]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook
output_notebook()
from bokeh.models import HoverTool,BoxZoomTool,ResetTool, WheelZoomTool
from bokeh.layouts import row, gridplot, layout, column
from bokeh.models.widgets import PreText, Select

Loading BokehJS ...

# Ressources

In [10]:
dfOmega=pd.read_json("https://raw.githubusercontent.com/grasshoff/geography/master/Geography/Omega.json")

In [11]:
dfXi=pd.read_json("https://raw.githubusercontent.com/grasshoff/geography/master/Geography/Xi.json")

# Preparation

In [12]:
dfOmega=dfOmega[['ID','type_sec','people','type','category','toponym','longitude_Omega','latitude_Omega']].set_index('ID')

In [13]:
dfXi=dfXi[['ID','type_sec','people','type','category','toponym','longitude_Xi','latitude_Xi']].set_index('ID')

# Filter

In [14]:
VettXi = dfXi[dfXi.people == 'Vettones']
VettOmega = dfOmega[dfOmega.people == 'Vettones']

In [20]:
dfVett = VettXi.merge(VettOmega[['longitude_Omega','latitude_Omega']], left_index=True, right_index=True)
dfVett['diff'] = np.where((dfVett['longitude_Omega']==dfVett['longitude_Xi']) & (dfVett['latitude_Omega']==dfVett['latitude_Xi']),'id','var')
dfVett

,type_sec,people,type,category,toponym,longitude_Xi,latitude_Xi,longitude_Omega,latitude_Omega,diff
ID,,,,,,,,,,
2.05.09.02,inland,Vettones,locality,city,Λακκιαοππιδανά,8.500000,41.666667,8.500000,41.666667,id
2.05.09.03,inland,Vettones,locality,city,Κοτταγοβρίγα,8.000000,41.500000,8.000000,41.500000,id
2.05.09.04,inland,Vettones,locality,city,Σαλμαντίκα,8.833333,41.833333,8.833333,41.833333,id
2.05.09.05,inland,Vettones,locality,city,Αὐγουστοβρίγα,8.000000,41.250000,8.000000,41.250000,id
2.05.09.06,inland,Vettones,locality,city,Ὄκελον,8.833333,40.750000,8.333333,41.250000,var
2.05.09.07,inland,Vettones,locality,city,Καπάρα,8.166667,41.000000,8.500000,41.000000,var
2.05.09.08,inland,Vettones,locality,city,Μανλιάνα,8.833333,41.000000,8.333333,41.000000,var
2.05.09.09,inland,Vettones,locality,city,Λακονιμουργί,8.333333,40.750000,8.333333,40.750000,id
2.05.09.10,inland,Vettones,locality,city,Δεοβρίγα,8.666667,40.666667,8.666667,40.666667,id


In [35]:
logV=(dfVett['longitude_Omega']==dfVett['longitude_Xi']) and (dfVett['latitude_Omega']==dfVett['latitude_Xi'])
dfVett['diff2']=dfVett.apply(lambda x: "id" if (logV) else 'var',axis=1)
dfVett

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Analysis
## Differences between Ξ and Ω

In [16]:
# Tooltip
vettones_var = dfVett.groupby('diff').get_group('var')
vettones_id = dfVett.groupby('diff').get_group('id')

# Xi 
n = figure(title= 'Vettones Ξ', width=350, height=450, x_range=(7.5, 9.5), y_range=(40, 42))
n.circle(np.array(dfVett["longitude_Xi"]),np.array(dfVett["latitude_Xi"]),fill_color='black',size=6,fill_alpha=0.4,line_color='black')
n.annulus(vettones_var["longitude_Xi"],vettones_var["latitude_Xi"],fill_color='green',inner_radius=0.04, outer_radius=0.06,fill_alpha=0.7,line_color='black',line_alpha=0)

# Omega 
o = figure(title= 'Vettones Ω', width=350, height=450, x_range=(7.5, 9.5), y_range=(40, 42))#,tools=toolsO)
o.circle(np.array(dfVett["longitude_Omega"]),np.array(dfVett["latitude_Omega"]),fill_color='black',size=6,fill_alpha=0.4,line_color='black')
o.annulus(vettones_var["longitude_Omega"],vettones_var["latitude_Omega"],fill_color='green',inner_radius=0.04, outer_radius=0.06,fill_alpha=0.7,line_color='black',line_alpha=0)

# Comparison
r = figure(title= 'Comparison', width=350, height=450, x_range=(7.5, 9.5), y_range=(40, 42))
r.circle(np.array(dfVett["longitude_Xi"]),np.array(dfVett["latitude_Xi"]),fill_color='red',size=6,fill_alpha=0.4,line_color='darkred')
r.circle(np.array(dfVett["longitude_Omega"]),np.array(dfVett["latitude_Omega"]),fill_color='blue',size=6,fill_alpha=0.4,line_color='darkblue')
r.circle(np.array(vettones_id["longitude_Omega"]),np.array(vettones_id["latitude_Omega"]),fill_color='grey',size=6.2,fill_alpha=1,line_color='grey')
r.segment(x0=dfVett["longitude_Xi"], y0=dfVett["latitude_Xi"], x1=dfVett["longitude_Omega"],
          y1=dfVett["latitude_Omega"], color="grey", line_width=1)

# Table
v2=dfVett.copy()
v2 = v2[['toponym']]
b = vettones_var
b2 = pd.merge(left=v2, right=b,how='outer')
b2 = b2.replace(to_replace = np.nan, value = "")
table = PreText(text="", width=750)
table.text = str(b2[['toponym','longitude_Omega','longitude_Xi','latitude_Omega','latitude_Xi']])


grid = gridplot([[n, o], [r, table]])
show(grid);

#### Result

Among the 11 localities of the Vettones, 3 have different coordinates in the two recensions: 
- Ocelum (longitude) 
- Capara (longitude)
- Manliana (longitude and latitude)

## Spatial ordering

Localities are linked on the map according to their ranking in the catalogue's list.

In [17]:
vettones_var = dfVett.groupby('diff').get_group('var')

# Xi Recension
n = figure(title= 'Vettones Ξ', width=350, height=450, x_range=(7.5, 9.5), y_range=(40, 42))
n.line(np.array(dfVett["longitude_Xi"]),np.array(dfVett["latitude_Xi"]),line_alpha=0.6,line_color='black')
n.circle(np.array(dfVett["longitude_Xi"]),np.array(dfVett["latitude_Xi"]),fill_color='grey',size=6,fill_alpha=1,line_color='black')
n.annulus(np.array(dfVett.groupby('diff').get_group('var')["longitude_Xi"]),np.array(dfVett.groupby('diff').get_group('var')["latitude_Xi"]),fill_color='green',inner_radius=0.04, outer_radius=0.06,fill_alpha=0.5,line_color='black',line_alpha=0)

# Omega Recension
o = figure(title= 'Vettones Ω', width=350, height=450, x_range=(7.5, 9.5), y_range=(40, 42))#,tools=toolsO)
o.line(np.array(dfVett["longitude_Omega"]),np.array(dfVett["latitude_Omega"]),line_alpha=0.6,line_color='black')
o.circle(np.array(dfVett["longitude_Omega"]),np.array(dfVett["latitude_Omega"]),fill_color='grey',size=6,fill_alpha=1,line_color='black')
o.annulus(np.array(dfVett.groupby('diff').get_group('var')["longitude_Omega"]),np.array(dfVett.groupby('diff').get_group('var')["latitude_Omega"]),fill_color='green',inner_radius=0.04, outer_radius=0.06,fill_alpha=0.5,line_color='black',line_alpha=0)

show(row(n,o));

#### Result

Two problematic areas can be identified: 

- the cities of Ocelum, Capara and Manliana have different coordinates in Ξ and Ω and the spatial ordering is disturbed in the Ξ recension

- the position of Salmantica, although it has the very same coordinates in Ξ and Ω, strays importantly from Ptolemy's ordering principle, which may indicate that a corruption occurred between Ptolemy's redaction and the archetype common to Ξ and Ω.

## Reconstruction 

The new dataframe contains the reconstructed coordinates. In a first step, we copy the reconstructed coordinates from the Xi recension. Then, these coordinates will be emended during the philological work. 

In [18]:
dfVettR = dfVett.copy()
dfVettR['longitude_r']=dfVettR['longitude_Xi']
dfVettR['latitude_r']=dfVettR['latitude_Xi']
dfVettR

,type_sec,people,type,category,toponym,longitude_Xi,latitude_Xi,longitude_Omega,latitude_Omega,diff,longitude_r,latitude_r
ID,,,,,,,,,,,,
2.05.09.02,inland,Vettones,locality,city,Λακκιαοππιδανά,8.500000,41.666667,8.500000,41.666667,id,8.500000,41.666667
2.05.09.03,inland,Vettones,locality,city,Κοτταγοβρίγα,8.000000,41.500000,8.000000,41.500000,id,8.000000,41.500000
2.05.09.04,inland,Vettones,locality,city,Σαλμαντίκα,8.833333,41.833333,8.833333,41.833333,id,8.833333,41.833333
2.05.09.05,inland,Vettones,locality,city,Αὐγουστοβρίγα,8.000000,41.250000,8.000000,41.250000,id,8.000000,41.250000
2.05.09.06,inland,Vettones,locality,city,Ὄκελον,8.833333,40.750000,8.333333,41.250000,var,8.833333,40.750000
2.05.09.07,inland,Vettones,locality,city,Καπάρα,8.166667,41.000000,8.500000,41.000000,var,8.166667,41.000000
2.05.09.08,inland,Vettones,locality,city,Μανλιάνα,8.833333,41.000000,8.333333,41.000000,var,8.833333,41.000000
2.05.09.09,inland,Vettones,locality,city,Λακονιμουργί,8.333333,40.750000,8.333333,40.750000,id,8.333333,40.750000
2.05.09.10,inland,Vettones,locality,city,Δεοβρίγα,8.666667,40.666667,8.666667,40.666667,id,8.666667,40.666667


#### Ocelum

Ocelum has a different longitude and a different latitude in the two recensions. The coordinates in the Ω recension are compatible with Ptolemy's ordering principle, whereas the Xi coordinates, in particular the latitude, stray away from the model. 

- The latitude in Xi is certainly corrupted whereas Omega offers the best reading.

- As for the longitude, both recensions supply an acceptable reading: Xi has the *lectio difficilior* ($L\gamma'$  Xi, $\gamma'$ Omega) but the reading $L\gamma'$ appears several times in the list before Ocelum, hence a miscopying in Xi would not be very surprising. In absence of a superior reading in one of the recension, I give priority to the Omega recension in this case, given that the reading for the latitude in Xi is very corrupted.

In [19]:
dfVettR.loc['2.05.09.06','latitude_r']=dfVettR.loc['2.05.09.06','latitude_Omega']
dfVettR.loc['2.05.09.06','longitude_r']=dfVettR.loc['2.05.09.06','longitude_Omega']

#### Capara and Manliana

Capara and Manliana are a more difficult problem as their respective longitudes are different in the two recensions but are compatible with Ptolemy's ordering principle. The divergence in the longitude of Manliana is a very common scribal issue: Omega has 30' ("L") whereas Xi has 10' ("ς"). Both signs are very close graphically and often mixed up. The divergence for Manliana is similar to the problem with Ocelum's longitude: Lγ in Xi and γ in Omega.

A sinlge element can help to decide. In the Omega recension the hand of the cartographer is supposed to situate first Capara, then Manliana: the second one is, however, to the left of Capara, which does not correspond to Ptolemy's instruction ("and the more western [localities] before the more eastern"). Since both cities in Omega are very close (10' of longitudinal interval), the configuration is not that discordant and could be acceptable. In absence of any other element, this situation in Omega tends to indicate that Xi has the best readings.

#### Salmantica

- Problem

The first four Vettonian localities have the same coordinates in both coordinates but show an important discordance with respect to Ptolemy's spatial ordering principle. This corruption can be explained if the city of Salmantica has erroneous coordinates, in this case, an erroneous latitude, already in the common archetype to Xi and Omega, hence in an early stage of transmission. An alternative scenario would be that both Lancia Oppidana and Cottaeobriga have corrupted latitudes.

- Vettones in context: the boundary between Lusitania and Tarraconensis

The Vettones are situated in the north-eastern part of the province Lusitania, near the boundary with the Tarraconensis province. On Ptolemy's map, the northern part of the boundary is formed by the Duero River while the eastern bourdary is defined by a line running through three points and settled on the Duero, Tagus and Anas rivers. The drawing of rivers in Ptolemy's *Geography* are not explicitely defined as straight lines; however, when a river does form an important curve, Ptolemy defined a point with coordinates that he registered in the catalogue, in order to make the particular drawing clear. In absence of indication, a river draws then a "more of less" straight line. 

The boundary point between Tarraconensis and Lusitania, situated at the Duero River, has very different coordinates in the two recensions. Its coordinates in the Omega recension are certainly corrupted as several localities of the Vettones would be situated in Tarraconensis instead of Lusitania. The Xi coordinates of this boundary point are satisfying but the very northern position of Salmantica is still slightly problematic as the city is to be found almost onto the river. A more southern position of Ptolemy's map would be certainly closer to Ptolemy's original version.

In [20]:
# drawing of the coast
g = dfXi[(dfXi.type_sec == 'coast section') & (dfXi.type == 'locality') & dfXi.category.apply(lambda row: row not in ['boundary','river path','river source'])][['longitude_Xi','latitude_Xi']]
h = dfOmega[(dfOmega.type_sec == 'coast section') & (dfOmega.type == 'locality') & dfOmega.category.apply(lambda row: row not in ['boundary','river path','river source'])][['longitude_Omega','latitude_Omega']]

p = figure(title="Xi Recension", x_axis_label='', y_axis_label='',plot_width=500, plot_height=500,x_range=(4.5,10.5), y_range=(40,43))
p.line(x=g['longitude_Xi'],y=g['latitude_Xi'],line_alpha=0.8, color='grey')
p.line([4.083,6.333,9,12], [37.667,39,39,37.25], line_alpha=0.8, color='grey')
p.line([5.333,9.333,9,9], [41.833,41.833,40.5,39], line_alpha=0.8, color='grey')
p.line([15.167,17,19,20.333], [45.833,43,43.167,42.333], line_alpha=0.8, color='grey')
p.circle(dfXi['longitude_Xi'],dfXi['latitude_Xi'],size=5, fill_color='grey', fill_alpha=0.5, line_color='grey',line_alpha=0)
p.line(dfVett['longitude_Xi'],dfVett['latitude_Xi'], line_color='navy',line_alpha=0.5)
p.circle(x=dfVett["longitude_Xi"],y=dfVett["latitude_Xi"],color='navy',size=5.5)
p.annulus(dfXi.loc['2.05.09.04',"longitude_Xi"],dfXi.loc['2.05.09.04',"latitude_Xi"],fill_color='green',inner_radius=0.06, outer_radius=0.1,fill_alpha=0.7,line_color='green',line_alpha=0)

q = figure(title="Omega Recension", x_axis_label='', y_axis_label='',plot_width=500, plot_height=500,x_range=(4.5,10.5), y_range=(40,43))
q.line(x=h['longitude_Omega'],y=h['latitude_Omega'],line_alpha=0.8, color='grey')
q.line([4.083,6.333,9,12], [37.667,39,39,37.25], line_alpha=0.8, color='grey')
q.line([5.333,9.167,9,9], [41.833,41.333,40.167,39], line_alpha=0.8, color='grey')
q.line([15,17,19,20.333], [45.833,43,43.167,42.333], line_alpha=0.8, color='grey')
q.circle(dfOmega['longitude_Omega'],dfOmega['latitude_Omega'],size=5, fill_color='grey', fill_alpha=0.5, line_color='grey',line_alpha=0)
q.line(dfVett['longitude_Omega'],dfVett['latitude_Omega'], line_color='navy',line_alpha=0.5)
q.circle(x=dfVett["longitude_Omega"],y=dfVett["latitude_Omega"],color='navy',size=5.5)
q.annulus(dfVett.loc['2.05.09.04',"longitude_Omega"],dfVett.loc['2.05.09.04',"latitude_Omega"],fill_color='green',inner_radius=0.06, outer_radius=0.1,fill_alpha=0.7,line_color='green',line_alpha=0)

show(row(p,q));

##### Proposition of conjecture

The latitude of Salmantica is 41°50' (μα Lγ'), where only the fraction part is certainly corrupted. We have already seen with the cases of Ocelum and Manliana that this number of minutes (Lγ) is involved in several scribal issues in the list of the Vettonian localities. If the latitude of Salmantica must respect Ptolemy's spatial ordering priciple, this city should be situated in latitude between Cottaeobriga and Augustobriga, that is, between 41°30' included and 41°20' included. In other words, the archetypal value was maybe L (30'), γιβ (25') or γ (20'). Readings L and γ are good candidates as they are graphically very close to Lγ and differ by only one sign from the "corrupted" value. I chose 30' because it is the most frequent value.

In [21]:
dfVettR.loc['2.05.09.04','latitude_r']=41.500

### Result

#### Plot

In [22]:
# Xi Recension
n = figure(title= 'Vettones Ξ', width=350, height=450, x_range=(7.5, 9.5), y_range=(40, 42))
n.line(np.array(dfVett["longitude_Xi"]),np.array(dfVett["latitude_Xi"]),line_alpha=0.6,line_color='black')
n.circle(np.array(dfVett["longitude_Xi"]),np.array(dfVett["latitude_Xi"]),fill_color='grey',size=6,fill_alpha=1,line_color='black')

# Omega Recension
o = figure(title= 'Vettones Ω', width=350, height=450, x_range=(7.5, 9.5), y_range=(40, 42))#,tools=toolsO)
o.line(np.array(dfVett["longitude_Omega"]),np.array(dfVett["latitude_Omega"]),line_alpha=0.6,line_color='black')
o.circle(np.array(dfVett["longitude_Omega"]),np.array(dfVett["latitude_Omega"]),fill_color='grey',size=6,fill_alpha=1,line_color='black')

# Reconstruction
p = figure(title= 'Vettones reconstruction', width=350, height=450, x_range=(7.5, 9.5), y_range=(40, 42))
p.line(np.array(dfVettR["longitude_r"]),np.array(dfVettR["latitude_r"]),line_alpha=0.6,line_color='black')
p.circle(np.array(dfVettR["longitude_r"]),np.array(dfVettR["latitude_r"]),fill_color='grey',size=6,fill_alpha=1,line_color='black')

show(row(n,o,p));

1)  between Ptolemy's original redaction and the archetype common to Xi and Omega: one modification, a corruption in the latitude of Salmantica

2) in the Xi recension, corruption of the coordinates of Ocelum 

3) in the Omega recension, corruption of the longitudes of Capara and Manliana.

#### Dataframe with reconstruction

In [23]:
dfVettR

,type_sec,people,type,category,toponym,longitude_Xi,latitude_Xi,longitude_Omega,latitude_Omega,diff,longitude_r,latitude_r
ID,,,,,,,,,,,,
2.05.09.02,inland,Vettones,locality,city,Λακκιαοππιδανά,8.500000,41.666667,8.500000,41.666667,id,8.500000,41.666667
2.05.09.03,inland,Vettones,locality,city,Κοτταγοβρίγα,8.000000,41.500000,8.000000,41.500000,id,8.000000,41.500000
2.05.09.04,inland,Vettones,locality,city,Σαλμαντίκα,8.833333,41.833333,8.833333,41.833333,id,8.833333,41.500000
2.05.09.05,inland,Vettones,locality,city,Αὐγουστοβρίγα,8.000000,41.250000,8.000000,41.250000,id,8.000000,41.250000
2.05.09.06,inland,Vettones,locality,city,Ὄκελον,8.833333,40.750000,8.333333,41.250000,var,8.333333,41.250000
2.05.09.07,inland,Vettones,locality,city,Καπάρα,8.166667,41.000000,8.500000,41.000000,var,8.166667,41.000000
2.05.09.08,inland,Vettones,locality,city,Μανλιάνα,8.833333,41.000000,8.333333,41.000000,var,8.833333,41.000000
2.05.09.09,inland,Vettones,locality,city,Λακονιμουργί,8.333333,40.750000,8.333333,40.750000,id,8.333333,40.750000
2.05.09.10,inland,Vettones,locality,city,Δεοβρίγα,8.666667,40.666667,8.666667,40.666667,id,8.666667,40.666667
